In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-5lgwxqti
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-5lgwxqti
  Resolved https://github.com/huggingface/transformers to commit 2e24ee4dfa39cc0bc264b89edbccc373c8337086
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.0 MB/s eta 0:00:0000:0100:01
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9922875 sha256=1a5f6e6cf351d4d8c8d126a355fcf81a55f5810cb262e6cd735bec5cb9e2a516
  Stored in directory: /tmp/pip-ephem-wheel-cache-0hcj2nkq/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Succ

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [3]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct"
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [4]:
from PIL import Image
import requests

In [7]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm  # Import tqdm for the progress bar

# Assuming you have already defined `processor` and `model` in your code

# Step 1: Define the folder containing images
image_folder = '/kaggle/input/facebook-hateful-meme-dataset/data/img'  # Replace with your Kaggle image folder path

# Step 2: Initialize an empty list to store the results
results = []

# Step 3: Get a sorted list of valid image files in ascending order
valid_images = sorted([image_name for image_name in os.listdir(image_folder) 
                       if image_name.endswith(('.png', '.jpg', '.jpeg'))])

# Step 4: Define start and end indices for processing images
start_index = 0
end_index = 1000

# Step 5: Process each image in the specified range
for index in tqdm(range(start_index, min(end_index, len(valid_images))), desc="Processing Images"):
    image_name = valid_images[index]  # Get the image name based on the current index
    image_path = os.path.join(image_folder, image_name)  # Construct the full image path
    
    # Open the image
    image = Image.open(image_path)

    # Prepare the messages for the model
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                },
                {
                    "type": "text",
                    "text": "Please extract the text from the image.",
                },
            ],
        }
    ]

    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt",
    )

    inputs = inputs.to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Replace \n with space
    output_text = [text.replace('\n', ' ') for text in output_text]

    # Prepare final output
    final_output = ' '.join(output_text)
    
    # Append the results with image name (id)
    image_id = os.path.splitext(image_name)[0]  # Get the image name without extension
    results.append({"id": image_id, "text": final_output})

# Step 6: Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Step 7: Save the DataFrame to a CSV file
output_csv_path = '/kaggle/working/extracted_texts.csv'  # Output CSV file path
results_df.to_csv(output_csv_path, index=False)

print(f"Text extraction completed. Results saved to {output_csv_path}.")


Processing Images: 100%|██████████| 5/5 [00:28<00:00,  5.63s/it]

Text extraction completed. Results saved to /kaggle/working/extracted_texts.csv.
